In [17]:
import os
from dataclasses import dataclass
from pathlib import Path
from cnnClassification.constants import *
from cnnClassification.utils.common import read_yaml_file, create_directories
import urllib.request as request
import zipfile
from cnnClassification.utils.common import get_size
from cnnClassification import logger

In [6]:
os.chdir("../")


In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path  
    # same as config file

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_configuration(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config

In [ ]:
# create components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zp_file(self):
        """
        zip_file_path : str
        Extract zp file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.mkdir(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref : 
            zip_ref.extractall(unzip_path)
                        
            
            
            
           